In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
--
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 13.144 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 1.345 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 1.622 seconds
--
DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.113 seconds
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.096 seconds
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;
Loading data to table default.tbl1
OK
Time taken: 1.005 seconds


In [3]:
%%hive 
SELECT * FROM tbl0;

SELECT * FROM tbl0;
OK
1	D	4	2016-06-25	["a","e","c","d"]	{"bb":10,"dd":20,"cc":40}
2	C	4	2015-05-14	["a","c"]	{"dd":40,"bb":20,"cc":10}
3	D	6	2014-12-26	["b","d"]	{"aa":10,"bb":40}
4	D	5	2016-06-25	["a","c","e","b","d"]	{"bb":40,"dd":20,"aa":10,"cc":30}
5	C	7	2016-05-23	["d","e","c"]	{"cc":10,"aa":20}
6	A	2	2018-06-14	["a","d"]	{"aa":20}
7	B	3	2014-12-22	["a","e","d"]	{"cc":20}
8	C	6	2015-08-20	["d","a","c","e"]	{"cc":10,"aa":20}
9	B	3	2017-12-08	["b","a","c","e"]	{"cc":40,"dd":10,"aa":30,"bb":20}
10	B	7	2015-07-28	["c","d","e","a","b"]	{"aa":10,"dd":40,"cc":30}
Time taken: 1.949 seconds, Fetched: 10 row(s)


In [4]:
%%hive 
SELECT c2, c6 FROM tbl0;

SELECT c2, c6 FROM tbl0;
OK
D	{"bb":10,"dd":20,"cc":40}
C	{"dd":40,"bb":20,"cc":10}
D	{"aa":10,"bb":40}
D	{"bb":40,"dd":20,"aa":10,"cc":30}
C	{"cc":10,"aa":20}
A	{"aa":20}
B	{"cc":20}
C	{"cc":10,"aa":20}
B	{"cc":40,"dd":10,"aa":30,"bb":20}
B	{"aa":10,"dd":40,"cc":30}
Time taken: 0.2 seconds, Fetched: 10 row(s)


In [35]:
%%hive
DROP TABLE IF EXISTS dataTable;
CREATE TABLE dataTable
AS
    SELECT
        c2, sumRow
    FROM (
        SELECT
            c2,
            COALESCE(c6['aa'], CAST(0 AS BIGINT)) + 
            COALESCE(c6['bb'], CAST(0 AS BIGINT)) + 
            COALESCE(c6['cc'], CAST(0 AS BIGINT)) + 
            COALESCE(c6['dd'], CAST(0 AS BIGINT)) AS sumRow
        FROM
            tbl0
    ) table1;

DROP TABLE IF EXISTS dataTable;
OK
Time taken: 0.127 seconds
CREATE TABLE dataTable
AS
    SELECT
        c2, sumRow
    FROM (
        SELECT
            c2,
            COALESCE(c6['aa'], CAST(0 AS BIGINT)) + 
            COALESCE(c6['bb'], CAST(0 AS BIGINT)) + 
            COALESCE(c6['cc'], CAST(0 AS BIGINT)) + 
            COALESCE(c6['dd'], CAST(0 AS BIGINT)) AS sumRow
        FROM
            tbl0
    ) table1;
Query ID = root_20200228123802_8c75f85a-1fd1-4ba9-bebe-916853234bd1
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1582886269318_0021, Tracking URL = http://7af61c813230:8088/proxy/application_1582886269318_0021/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582886269318_0021
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-28 12:38:09,938 Stage-1 map = 0%,  reduce = 0%
2020-02-28 12:38:16,658 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU

In [38]:
%%hive
SELECT * FROM dataTable;

SELECT * FROM dataTable;
OK
D	70
C	70
D	50
D	100
C	30
A	20
B	20
C	30
B	100
B	80
Time taken: 0.379 seconds, Fetched: 10 row(s)


In [40]:
%%hive 
DROP TABLE IF EXISTS SumDT;
CREATE TABLE SumDT
AS
    SELECT c2, SUM(sumRow)
    FROM
        dataTable
GROUP BY
    c2;

DROP TABLE IF EXISTS SumDT;
OK
Time taken: 0.007 seconds
CREATE TABLE SumDT
AS
    SELECT c2, SUM(sumRow)
    FROM
        dataTable
GROUP BY
    c2;
Query ID = root_20200228124001_1d075190-c367-4743-9348-e88a3325f181
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1582886269318_0022, Tracking URL = http://7af61c813230:8088/proxy/application_1582886269318_0022/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582886269318_0022
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-02-28 12:40:08,982 Stage-1 map = 0%,  reduce = 0%
2020-02-28 12:40:15,602 Stage-1 map = 100%,  reduce = 

In [42]:
%%hive
SELECT * FROM SumDT;

SELECT * FROM SumDT;
OK
A	20
B	200
C	130
D	220
Time taken: 0.126 seconds, Fetched: 4 row(s)


In [43]:
%hive_quit

In [44]:
## Copia Archivos a sistema HDFS
##
## Se usan un directorio temporal en el HDFS. La siguiente
## instrucción muestra el contenido del dicho directorio
##
!hdfs dfs -ls /tmp

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-28 10:38 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-28 10:51 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-28 12:14 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-28 11:35 /tmp/wordcount


In [45]:
## Crea la carpeta wordcount en el hdfs
##
!hdfs dfs -mkdir /tmp/wordcount

mkdir: `/tmp/wordcount': File exists


In [46]:
## Verifica la creación de la carpeta
##
!hdfs dfs -ls /tmp/

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-28 10:38 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-28 10:51 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-28 12:14 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-28 11:35 /tmp/wordcount


In [47]:
## Copia los archvios del directorio local wordcount/
## al directorio /tmp/wordcount/ en el hdfs
##
!hdfs dfs -copyFromLocal 'data.tsv'  /tmp/wordcount/

copyFromLocal: `data.tsv': No such file or directory


In [48]:
##
## Verifica que los archivos esten copiados
## en el hdfs
##
!hdfs dfs -ls /tmp/wordcount

In [49]:
####################################################################################################################
## Generación del script y ajuste del código

In [52]:
%%writefile wordcount.hql

DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
--
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

DROP TABLE IF EXISTS dataTable;
CREATE TABLE dataTable
AS
    SELECT
        c2, sumRow
    FROM (
        SELECT
            c2,
            COALESCE(c6['aa'], CAST(0 AS BIGINT)) + 
            COALESCE(c6['bb'], CAST(0 AS BIGINT)) + 
            COALESCE(c6['cc'], CAST(0 AS BIGINT)) + 
            COALESCE(c6['dd'], CAST(0 AS BIGINT)) AS sumRow
        FROM
            tbl0
    ) table1;
    
DROP TABLE IF EXISTS SumDT;
CREATE TABLE SumDT
AS
    SELECT c2, SUM(sumRow)
    FROM
        dataTable
GROUP BY
    c2;
    
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','

SELECT * FROM SumDT;

Overwriting wordcount.hql


In [53]:
## Ejecucion
!hive -S -e 'source wordcount.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


In [54]:
### Copia de los resultados a la maquina local (vagrant)
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [55]:
!cat output/000000_0 | head

A,20
B,200
C,130
D,220
